# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235595008085                   -0.50    7.0   10.7ms
  2   -7.250418659775       -1.83       -1.40    1.0   5.44ms
  3   -7.251217673621       -3.10       -2.27    1.0   5.23ms
  4   -7.251325664533       -3.97       -2.44    2.0   6.47ms
  5   -7.251335394217       -5.01       -2.86    1.0   5.32ms
  6   -7.251337412487       -5.70       -3.05    2.0   6.28ms
  7   -7.251338724626       -5.88       -3.63    1.0   5.52ms
  8   -7.251338783158       -7.23       -3.98    1.0   5.66ms
  9   -7.251338795988       -7.89       -4.31    1.0   5.67ms
 10   -7.251338798125       -8.67       -4.59    2.0   45.8ms
 11   -7.251338798520       -9.40       -4.90    1.0   6.19ms
 12   -7.251338798647       -9.90       -5.16    2.0   7.27ms
 13   -7.251338798697      -10.29       -5.75    1.0   6.06ms
 14   -7.251338798702      -11.35       -5.81    3.0   7.96ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.050408485829918366
[ Info: Arnoldi iteration step 2: normres = 0.46790047772085686
[ Info: Arnoldi iteration step 3: normres = 0.9698175755209208
[ Info: Arnoldi iteration step 4: normres = 0.2868478211907985
[ Info: Arnoldi iteration step 5: normres = 0.6068282042073576
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.40e-02, 8.69e-02, 5.68e-01, 1.86e-01, 4.70e-03)
[ Info: Arnoldi iteration step 6: normres = 0.2452895635346112
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.26e-02, 1.30e-01, 1.42e-01, 1.05e-01, 8.98e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06982931943631544
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.32e-04, 8.83e-03, 8.26e-03, 5.28e-02, 3.66e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09137492729964601
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.69e